In [1]:
#Load up the required packages
library(phyloseq)
library(ggplot2)
library(vegan)
library(reshape2)
library(stringr)
library(ape)

library(dplyr)
library(data.table)

sessionInfo()

Loading required package: permute

Loading required package: lattice

This is vegan 2.5-5


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following objects are masked from ‘package:reshape2’:

    dcast, melt




R version 3.5.2 (2018-12-20)
Platform: x86_64-redhat-linux-gnu (64-bit)
Running under: Fedora 28 (Workstation Edition)

Matrix products: default
BLAS/LAPACK: /usr/lib64/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_GB.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_GB.UTF-8        LC_COLLATE=en_GB.UTF-8    
 [5] LC_MONETARY=en_GB.UTF-8    LC_MESSAGES=en_GB.UTF-8   
 [7] LC_PAPER=en_GB.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_GB.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] data.table_1.12.2 dplyr_0.8.4       ape_5.3           stringr_1.4.0    
 [5] reshape2_1.4.3    vegan_2.5-5       lattice_0.20-38   permute_0.9-5    
 [9] ggplot2_3.2.1     phyloseq_1.26.1  

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-3        tidyselect_0.2.5    repr_1.0.1         
 [4] purrr_0.3.3     

In [2]:
#Assuming the data is conatined in files with names matching *.biom.json 
biomlist <- list.files('Inputs/biom_files/', '*.json', full.names = T)
prj_names <- gsub('\\.biom\\.json','\\1',basename(biomlist))
nmgsdirs <- paste('./NMGS_processing/',prj_names, sep="")

In [5]:
#Only required if performaing NMGS theta calculations
for (p in nmgsdirs)
#Create the base NMGS directory
    dir.create(p, recursive = T)


In [6]:
classify_data <- read.csv('Inputs/Taxa_Classifications_20191218.csv', header = T, comment.char = '#', stringsAsFactors = F )
metabolic_energies <- read.csv('Inputs/Energies_with_RET.csv')

In [7]:
met_paths <- unique(classify_data$Metabolism)
names(met_paths) <- met_paths

#Build up Taxa search expressions for them
Taxa_Search_Expressions <- lapply(met_paths, FUN = function(n){
  X = classify_data[classify_data$Metabolism==n,]
  d <- apply(X, 1, function(X) paste(X[[2]],'=="',X[[1]],'"',sep=''))
  searchstr  = paste(unlist(d),collapse = ' | ')
  parse(text=searchstr)
})


In [8]:
PSobjs_full <- sapply(biomlist, function(x) import_biom(x, parseFunction = parse_taxonomy_greengenes))

Warning message in parseFunction(i$metadata$taxonomy):
“No greengenes prefixes were found. 
Consider using parse_taxonomy_default() instead if true for all OTUs. 
Dummy ranks may be included among taxonomic ranks now.”
Warning message in parseFunction(i$metadata$taxonomy):
“No greengenes prefixes were found. 
Consider using parse_taxonomy_default() instead if true for all OTUs. 
Dummy ranks may be included among taxonomic ranks now.”
Warning message in parseFunction(i$metadata$taxonomy):
“No greengenes prefixes were found. 
Consider using parse_taxonomy_default() instead if true for all OTUs. 
Dummy ranks may be included among taxonomic ranks now.”
Warning message in parseFunction(i$metadata$taxonomy):
“No greengenes prefixes were found. 
Consider using parse_taxonomy_default() instead if true for all OTUs. 
Dummy ranks may be included among taxonomic ranks now.”


In [9]:
PSobjs_full

$`Inputs/biom_files//AmericanGut_ERP012803.biom.json`
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 12886 taxa and 462 samples ]
tax_table()   Taxonomy Table:    [ 12886 taxa by 7 taxonomic ranks ]

$`Inputs/biom_files//Atlantic_ERP012887.biom.json`
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15667 taxa and 385 samples ]
tax_table()   Taxonomy Table:    [ 15667 taxa by 7 taxonomic ranks ]

$`Inputs/biom_files//CanadaWater_ERP012927.biom.json`
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 23828 taxa and 388 samples ]
tax_table()   Taxonomy Table:    [ 23828 taxa by 7 taxonomic ranks ]

$`Inputs/biom_files//Donna_ADSludge.biom.json`
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 11585 taxa and 22 samples ]
tax_table()   Taxonomy Table:    [ 11585 taxa by 7 taxonomic ranks ]

$`Inputs/biom_files//DuckWater_ERP012631.biom.json`
phyloseq-class experiment-level object
otu_table() 

In [10]:
PSobjs_rare <- sapply(names(PSobjs_full), function(n){
    ps <- PSobjs_full[[n]]
    h = hist(sample_sums(ps), breaks=40, plot=F)
    top_80 = which(cumsum(h$counts) / sum(h$counts) > 0.2)[[1]]
    rarefy_even_depth(ps, sample.size = h$mids[[top_80]])
    })


You set `rngseed` to FALSE. Make sure you've set & recorded
 the random seed of your session for reproducibility.
See `?set.seed`


...

104 samples removedbecause they contained fewer reads than `sample.size`.

Up to first five removed samples are: 


ERR1090786ERR1090787ERR1090789ERR1090812ERR1090820	

...

2044OTUs were removed because they are no longer 
present in any sample after random subsampling


...

You set `rngseed` to FALSE. Make sure you've set & recorded
 the random seed of your session for reproducibility.
See `?set.seed`


...

74 samples removedbecause they contained fewer reads than `sample.size`.

Up to first five removed samples are: 


ERR1102990ERR1102991ERR1102992ERR1102993ERR1102994	

...

3644OTUs were removed because they are no longer 
present in any sample after random subsampling


...

You set `rngseed` to FALSE. Make sure you've set & recorded
 the random seed of your session for reproducibility.
See `?set.seed`


...

84 samples removedbecause they con

In [11]:
#Named list for all results
all_filtered_list = list()

for (i in seq(length(PSobjs_full))){
    curbiom = PSobjs_full[[i]]
    filtered_list <- lapply(Taxa_Search_Expressions, function(x){
                                return(tryCatch(do.call(subset_taxa, list(curbiom,x)),error=function(e) NULL))
                            })
    filtered_list = filtered_list[!sapply(filtered_list, is.null)]
    all_filtered_list[[prj_names[[i]]]] <- filtered_list
                        
}

In [12]:
#Named list for all results
all_filtered_rare = list()

for (i in seq(length(PSobjs_rare))){
    curbiom = PSobjs_rare[[i]]
    filtered_list <- lapply(Taxa_Search_Expressions, function(x){
                                return(tryCatch(do.call(subset_taxa, list(curbiom,x)),error=function(e) NULL))
                            })
    filtered_list = filtered_list[!sapply(filtered_list, is.null)]
    all_filtered_rare[[prj_names[[i]]]] <- filtered_list
                        
}

In [13]:
allprojects <- list()
genusprojects <- list()
for (i in seq(length(all_filtered_list))){
    raw = PSobjs_full[[i]]
    genus_raw  = tax_table(raw)[!is.na(tax_table(raw)[,"Genus"]),]
    flt = all_filtered_list[[i]]
    allprojects[[names(all_filtered_list)[[i]]]] = sum(sapply(flt, function(x) sum(otu_table(x)))) / sum(otu_table(raw)) * 100
    genusprojects[[names(all_filtered_list)[[i]]]] = sum(sapply(flt, function(x) sum(otu_table(x)))) / sum(otu_table(raw)[rownames(genus_raw),]) * 100    
}
t(rbind(as.data.frame(allprojects), as.data.frame(genusprojects)))

AmericanGut_ERP012803,96.72597,174.36957
Atlantic_ERP012887,41.21598,117.35322
CanadaWater_ERP012927,39.28345,98.27441
Donna_ADSludge,54.11863,116.13445
DuckWater_ERP012631,87.92439,117.15786
Hydrothermal_ERP011826,23.43172,228.57385
Lise_Soils,23.76604,100.39162
MalawianChildren_ERP005437,98.48448,127.11381
Marine_ERP013553,66.59707,117.96768
MexicanSoil_SRP037963,39.47895,149.64725
ParkGrass_SRP044877,23.14114,92.71557


In [14]:
#Generate NMGS input files

for (i in seq(length(all_filtered_list))){
    curbiom <- all_filtered_list[[i]]
    for (x in names(curbiom)){
        outfname = paste(nmgsdirs[[i]],'/',x,'.csv', sep = '')
        write.csv(curbiom[[x]]@otu_table, outfname)
    }
}

For efficiency, the actuall NMGS jobs are run on a seperate HPC system, using the following Grid Engine submission script:

``` bash
#!/bin/bash
 
#$ -pe smp 4
#$ -cwd -V
#$ -q all.q,bigmem.q
#$ -l h_vmem=16g
 
fname=$1
 
bname=`basename $fname`
 
module add libs/gsl
 
if [ ! -e out_$bname.csv.csv ]
then
        ~/NMGS/C/NMGS -in $fname -out out_$bname -v -nt $NSLOTS -lm
fi
 
```



And the following code is used to pre-process the huge NMGS outputs


``` R
library(plyr)
library(NMGS)

dirlist = gsub('\\.\\/\\/?','',list.dirs(path = './', ))[-1]

allsets <- list()


for (dir in dirlist){
  curset <- list()
  curset\$dirname = dir
  for (f in list.files(dir, 'out*')){
    nmgs_res <- read_nmgs(paste(dir,f,sep='/'))
    theta <- nmgs_posterior_summaries(nmgs_res, burnin = 25000, thinning = 10)["theta","median"]
    type = gsub("out_(.*)\\.csv\\.csv","\\1",f)
    curset[type] = theta
  }
  save(curset, file=paste('results_',dir,'.RData', sep=""))
  allsets[[dir]] = curset
}


save(allsets, file="allsets.save.RData")

flist = list.files('./', 'results.*')
prjlist = gsub('results_(.*)\\.RData','\\1',flist)
for (f in flist){
  load(f)
  allsets[[gsub('results_(.*)\\.RData','\\1',f)]] <- curset
}

fullresults <- rbind.fill(lapply(allsets, data.frame))

fullresults[fullresults==0]=NA

fullresults\$biome = c("Human","Marine","Freshwater","WWT","Freshwater","Volcanic","Marine", "Human","Marine","Soil","Soil","WWT","Marine", "Volcanic")
fullresults\$sample = c("American Gut","Atlantic", "Canadian lakes", "Anaerobic sludge", "Farm water", "Hydrothermal vent", "Arctic soil", "Malawian children", "Baltic", "Mexican soil","Canadian parks", "Severn Trent WWT", "TARA Ocean Survey", "Italian Volcanoes")

fullresults
write.table(fullresults, 'metacommunity_diversity_20180703.txt',sep="\t");




 ```

In [72]:
#Set up the plot axes - for log scales
xbreaks = c(seq(0.001,0.01,0.001),seq(0.01,0.1,0.01), seq(0.1,1,0.1), seq(1,10,1), seq(10,100,10), seq(100,1000,100),seq(1000,10000,1000))
xlabs <- c("0.001",rep("",9),"0.01",rep("",9),"0.1",rep("",9),"1",rep("",9), "10", rep("",9), "100", rep("",9), "1000", rep("",9))

ybreaks = c(seq(0.1,1,0.1),seq(1,10,1), seq(10,100,10), seq(100,1000,100), seq(1000,10000,1000))
ylabs <- c("0.1",rep("",9),"1",rep("",9),"10",rep("",9),"100",rep("",9),"1000",rep("",9))

#Metabolic colours
metab_color_list <-  c("wheat3", "blue","red", "chartreuse3", "darkgoldenrod2","orange","darkred", "purple", "darkgreen", "yellow4", "steelblue3","firebrick2", "indianred", "darkolivegreen4", "honeydew4")
names(metab_color_list) <- metabolic_energies$Metabolism
metab_label_list = as.character(metabolic_energies$Name)
names(metab_label_list) <- names(metab_color_list)


#Per study labels and colors
study_info_df <- data.frame(
    study=c('AmericanGut_ERP012803', 'Atlantic_ERP012887', 'CanadaWater_ERP012927', 'Donna_ADSludge', 'DuckWater_ERP012631', 'Hydrothermal_ERP011826', 'Lise_Soils', 'MalawianChildren_ERP005437', 'Marine_ERP013553', 'MexicanSoil_SRP037963', 'ParkGrass_SRP044877', 'SevernTrent', 'TARA_ERP001736', 'Volcanic_ERP010094'),
    biome = c("Human","Marine","Freshwater","WWT","Freshwater","Volcanic","Soil", "Human","Marine","Soil","Soil","WWT","Marine", "Volcanic"),
    studylabel = c("American Gut","Atlantic", "Canadian lakes", "Anaerobic sludge", "Farm water", "Hydrothermal vent", "Arctic soil", "Malawian children", "Baltic", "Mexican soil","Canadian parks", "Severn Trent WWT", "TARA Ocean Survey", "Italian Volcanoes")
)

study_name_list = as.character(study_info_df$studylabel)
names(study_name_list) <- as.character(study_info_df$study)
study_biome_list = as.character(study_info_df$biome)
names(study_biome_list) <- as.character(study_info_df$study)
biome_color_list <- c("salmon3","darkseagreen","cornflowerblue","bisque3","darkorange","darkslategray")
names(biome_color_list) <- c("Human", "Marine", "Freshwater", "WWT", "Volcanic", "Soil")


#Carbon source colors + labels
c_src_colours = c("blue","orange")
names(c_src_colours) = c("C","S")

c_src_labels = c("Complex","Simple")
names(c_src_colours) = c("C","S")


In [73]:
#Marine_ERP013553 doesn't have enough diversity per sample (coverage is too low)
all_filtered_list[["Marine_ERP013553"]] <- NULL

In [74]:
alpha_df_list <- lapply(names(all_filtered_list), function(curname){
    curPrj <- all_filtered_list[[curname]]
    
    prj_diversity_df <- do.call(rbind,
            lapply(names(curPrj), function(b){
                df <- estimate_richness(curPrj[[b]], measures = c("Observed","InvSimpson","Shannon", "Chao1"))
                df$metabolism <- b
                df$sample <- rownames(df)
                df$study <- curname
                df
            }))
    prj_diversity_df
})

Warning message in sqrt(sum(Deriv.Ch1 %*% t(Deriv.Ch1) * (diag(a) - a %*% t(a)/S.ACE))):
“NaNs produced”
Warning message in estimate_richness(curPrj[[b]], measures = c("Observed", "InvSimpson", :
“The data you have provided does not have
any singletons. This is highly suspicious. Results of richness
estimates (for example) are probably unreliable, or wrong, if you have already
trimmed low-abundance taxa from the data.

We recommended that you find the un-trimmed data and retry.”
Warning message in estimate_richness(curPrj[[b]], measures = c("Observed", "InvSimpson", :
“The data you have provided does not have
any singletons. This is highly suspicious. Results of richness
estimates (for example) are probably unreliable, or wrong, if you have already
trimmed low-abundance taxa from the data.

We recommended that you find the un-trimmed data and retry.”
Warning message in estimate_richness(curPrj[[b]], measures = c("Observed", "InvSimpson", :
“The data you have provided does not have
any 

In [75]:
#combine all diversities into a single data.frame
alpha_diversity = do.call(rbind,alpha_df_list)

In [76]:
#fixup columns 
#Drop "inf" values - not all samples from all projects contain examples from all metabolisms
alpha_diversity <- alpha_diversity[is.finite(alpha_diversity$InvSimpson),]

#and merge in the metabolic values:
alpha_diversity = merge(alpha_diversity, metabolic_energies, by.x="metabolism", by.y="Metabolism")
      

In [77]:
colnames(alpha_diversity)[colnames(alpha_diversity) == "sample"] <- "samp_name"

In [78]:
alpha_diversity  %>% select(metabolism, InvSimpson, samp_name, study) %>% 
    group_by(samp_name) %>% arrange(desc(InvSimpson)) %>% #summarise(metab=list(metabolism), )
    mutate(ord = InvSimpson / max(InvSimpson)) %>% group_by(metabolism) -> tmpdf

In [79]:
tmpdf %>% group_by(metabolism) %>% summarise(m = mean(ord)) %>% arrange(desc(m))

metabolism,m
<chr>,<dbl>
AerobicResp,0.67755666
Ferm,0.65206845
CH3_Methanogen,0.32457440
SRB,0.22396611
FeIII_reducer,0.15037003
Methanotroph,0.14999334
NOB,0.14069482
Ammonia_Oxidizer,0.13175282
H2_Oxidizer,0.10423541


In [80]:
#Calulate per-sample linear best fits
alpha_fits_yield <- alpha_diversity %>% group_by(study, samp_name) %>% do(
    as.data.frame(summary(lm(log(InvSimpson) ~ log(yield), data=.))$coefficients, stringsAsFactors = F))


alpha_fits_dgcat <- alpha_diversity %>% group_by(study, samp_name) %>% do(
    as.data.frame(summary(lm(log(InvSimpson) ~ log(-dgcat), data=.))$coefficients, stringsAsFactors = F))

In [81]:
alpha_fits_yield

study,samp_name,Estimate,Std. Error,t value,Pr(>|t|)
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
AmericanGut_ERP012803,ERR1090777,1.09022348,0.6334567,1.7210701,0.1837209
AmericanGut_ERP012803,ERR1090777,0.03123958,0.2537115,0.1231303,0.9097898
AmericanGut_ERP012803,ERR1090778,1.22646711,0.5694563,2.1537511,0.1202817
AmericanGut_ERP012803,ERR1090778,0.16051670,0.2280781,0.7037795,0.5322772
AmericanGut_ERP012803,ERR1090779,1.61021800,0.8060495,1.9976663,0.1838214
AmericanGut_ERP012803,ERR1090779,0.12965077,0.3160637,0.4102045,0.7214238
AmericanGut_ERP012803,ERR1090780,1.49364580,0.6673099,2.2383090,0.1111358
AmericanGut_ERP012803,ERR1090780,0.15866693,0.2672703,0.5936571,0.5945267
AmericanGut_ERP012803,ERR1090781,1.39873306,0.7568646,1.8480625,0.1617358


In [82]:
#Finally, group all samples together and generate the "full" fit across tall samples in a project
alpha_fullfits_yield <- alpha_diversity %>% group_by(study) %>% do(as.data.frame(summary(lm(log(InvSimpson) ~ log(yield), data=.))$coefficients, stringsAsFactors = F))
#Finally, group all samples together and generate the "full" fit across tall samples in a project
alpha_fullfits_dgcat <- alpha_diversity %>% group_by(study) %>% do(as.data.frame(summary(lm(log(InvSimpson) ~ log(-dgcat), data=.))$coefficients, stringsAsFactors = F))

In [83]:
#Relabel the parameters (intercept and slope) that were lost "do" aglomeration
alpha_fits_yield$param = c("intercept","slope")
alpha_fullfits_yield$param = c("intercept","slope")
alpha_fits_dgcat$param = c("intercept","slope")
alpha_fullfits_dgcat$param = c("intercept","slope")

#Reshape the dataframes
alpha_fitsdf_yield <- melt(alpha_fits_yield, id.vars = c("study","samp_name","param")) %>% dcast(study + samp_name ~ param+variable, value.var = "value")
alpha_fullfitsdf_yield <- melt(alpha_fullfits_yield, id.vars = c("study","param")) %>% dcast(study ~ param+variable, value.var = "value")

alpha_fitsdf_dgcat <- melt(alpha_fits_dgcat, id.vars = c("study","samp_name","param")) %>% dcast(study + samp_name ~ param+variable, value.var = "value")
alpha_fullfitsdf_dgcat <- melt(alpha_fullfits_dgcat, id.vars = c("study","param")) %>% dcast(study ~ param+variable, value.var = "value")

rownames(alpha_fullfitsdf_yield) <- alpha_fullfitsdf_yield$study
rownames(alpha_fullfitsdf_dgcat) <- alpha_fullfitsdf_dgcat$study

In [89]:
alpha_fullfitsdf_dgcat

,study,intercept_Estimate,intercept_Std. Error,intercept_t value,intercept_Pr(>|t|),slope_Estimate,slope_Std. Error,slope_t value,slope_Pr(>|t|)
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AmericanGut_ERP012803,AmericanGut_ERP012803,-0.17882382,0.08112370,-2.2043351,2.759751e-02,0.2462478,0.01437166,17.134257,4.212810e-62
Atlantic_ERP012887,Atlantic_ERP012887,-1.31057353,0.08629568,-15.1870113,7.085050e-50,0.3947365,0.01381362,28.575883,1.787862e-155
CanadaWater_ERP012927,CanadaWater_ERP012927,0.03376165,0.06188617,0.5455443,5.854087e-01,0.2342817,0.01075755,21.778360,1.230058e-99
Donna_ADSludge,Donna_ADSludge,-0.40170332,0.20366256,-1.9723965,4.956607e-02,0.2523358,0.03600500,7.008355,1.861981e-11
DuckWater_ERP012631,DuckWater_ERP012631,-2.23049868,0.10209504,-21.8472769,2.096339e-93,0.4934708,0.01616063,30.535375,1.803696e-163
Hydrothermal_ERP011826,Hydrothermal_ERP011826,-0.52233949,0.20206297,-2.5850332,1.058609e-02,0.2321522,0.03741589,6.204642,4.130334e-09
Lise_Soils,Lise_Soils,-0.28431525,0.71697368,-0.3965491,6.947062e-01,0.2201198,0.12211596,1.802547,8.223407e-02
MalawianChildren_ERP005437,MalawianChildren_ERP005437,1.21507634,0.25368121,4.7897767,1.928045e-06,0.0509102,0.03522960,1.445097,1.487509e-01
MexicanSoil_SRP037963,MexicanSoil_SRP037963,-0.36417662,0.41538145,-0.8767282,3.817837e-01,0.3666187,0.06823557,5.372838,2.336493e-07


In [92]:
unique(alpha_fitsdf_dgcat$study)

[1] "AmericanGut_ERP012803"      "Atlantic_ERP012887"        
 [3] "CanadaWater_ERP012927"      "Donna_ADSludge"            
 [5] "DuckWater_ERP012631"        "Hydrothermal_ERP011826"    
 [7] "Lise_Soils"                 "MalawianChildren_ERP005437"
 [9] "MexicanSoil_SRP037963"      "ParkGrass_SRP044877"       
[11] "SevernTrent"                "TARA_ERP001736"            
[13] "Volcanic_ERP010094"

In [85]:
for (curstudy in unique(alpha_fitsdf_dgcat$study)){

    plot_df <- alpha_diversity[alpha_diversity$study == curstudy,]
    
    fits_df_yield <- alpha_fitsdf_yield[alpha_fitsdf_yield$study == curstudy,]
    fits_df_dgcat <- alpha_fitsdf_dgcat[alpha_fitsdf_dgcat$study == curstudy,]



    p_yield <- ggplot(plot_df, aes(x=yield, y=InvSimpson)) +
        theme_bw(base_size = 18, base_family = "arial") +

        #geom_abline(data=fits_df_yield[fits_df_yield$`slope_Pr(>|t|)` <= 0.05,], 
        #              aes(slope=slope_Estimate, intercept=intercept_Estimate), alpha=0.05, color="black") +
        #geom_abline(data=fitdf[fitdf$`slope_Pr(>|t|)` > 0.05,], 
        #              aes(slope=slope_Estimate, intercept=intercept_Estimate), alpha=0.05, color="black") +


        geom_abline(data=fits_df_yield, aes(slope=slope_Estimate, intercept=intercept_Estimate), alpha=0.05, color="black") +

        geom_point(size=5, aes(color=metabolism), alpha=1.0) +
        geom_point(size=2, color="LightBlue", alpha=0.8) + 

        scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
        scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

        scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

        xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
        ylab(expression(alpha ~ "Diversity (inverse Simpson)"))  +
        geom_abline(slope=alpha_fullfits_yield[curstudy, "slope_Estimate"], intercept = alpha_fullfits_yield[curstudy, "intercept_Estimate"],
                 alpha = 1.0, color="darkgreen", size=2) +
        ggtitle(study_name_list[curstudy]) 


    p_catab <- ggplot(plot_df, aes(x=-dgcat, y=InvSimpson)) +
        theme_bw(base_size = 18, base_family = "arial") +

        #geom_abline(data=fits_df_yield[fits_df_yield$`slope_Pr(>|t|)` <= 0.05,], 
        #              aes(slope=slope_Estimate, intercept=intercept_Estimate), alpha=0.05, color="black") +
        #geom_abline(data=fitdf[fitdf$`slope_Pr(>|t|)` > 0.05,], 
        #              aes(slope=slope_Estimate, intercept=intercept_Estimate), alpha=0.05, color="black") +


        geom_abline(data=fits_df_dgcat, aes(slope=slope_Estimate, intercept=intercept_Estimate), alpha=0.05, color="black") +

        geom_point(size=5, aes(color=metabolism), alpha=1.0) +
        geom_point(size=2, color="LightBlue", alpha=0.8) + 

        scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
        scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

        scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +
        xlab(expression(-Delta*G[cat](kJmol^-1))) +

        ylab(expression(alpha ~ "Diversity (inverse Simpson)"))  +
        geom_abline(slope=alpha_fullfits_dgcat[curstudy, "slope_Estimate"], intercept = alpha_fullfits_dgcat[curstudy, "intercept_Estimate"],
                 alpha = 1.0, color="darkgreen", size=2) +
        ggtitle(study_name_list[curstudy]) 


    ggsave(paste('output_images/alpha/yield-',curstudy,'.png', sep=""),p_yield,dpi = 600, 
               units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 

    ggsave(paste('output_images/alpha/catab-',curstudy,'.png', sep=""),p_catab,dpi = 600, 
               units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 

}

ERROR: Error: Can't find column `slope_Estimate` in `.data`.


In [91]:
alpha_fullfits_yield[curstudy,]

study,Estimate,Std. Error,t value,Pr(>|t|),param
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
NA,NA,NA,NA,NA,NA


In [88]:
fits_df_dgcat

study,samp_name,intercept_Estimate,intercept_Std. Error,intercept_t value,intercept_Pr(>|t|),slope_Estimate,slope_Std. Error,slope_t value,slope_Pr(>|t|)
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AmericanGut_ERP012803,ERR1090777,0.9423341,2.019023,0.46672787,0.6724991,0.02067529,0.3553608,0.05818113,0.9572629
AmericanGut_ERP012803,ERR1090778,-0.1456611,1.819209,-0.08006840,0.9412250,0.21850411,0.3201923,0.68241530,0.5439459
AmericanGut_ERP012803,ERR1090779,0.6975431,3.046331,0.22897814,0.8401695,0.13741896,0.5034295,0.27296563,0.8104821
AmericanGut_ERP012803,ERR1090780,0.1423014,2.130202,0.06680182,0.9509423,0.21507443,0.3749291,0.57364031,0.6063932
AmericanGut_ERP012803,ERR1090781,-0.3012734,2.368430,-0.12720380,0.9068264,0.27597225,0.4168588,0.66202813,0.5552656
AmericanGut_ERP012803,ERR1090782,0.2190173,1.731413,0.12649633,0.9073410,0.09398756,0.3047396,0.30841927,0.7779391
AmericanGut_ERP012803,ERR1090783,0.8171809,2.705659,0.30202653,0.7911448,0.09832956,0.4471310,0.21991220,0.8463452
AmericanGut_ERP012803,ERR1090784,0.3719168,2.327639,0.15978290,0.8832041,0.11315647,0.4096792,0.27620753,0.8003237
AmericanGut_ERP012803,ERR1090785,0.3775121,2.347939,0.16078443,0.8824803,0.11246579,0.4132522,0.27214806,0.8031638


In [87]:
alpha_fullfits_yield

study,Estimate,Std. Error,t value,Pr(>|t|),param
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
AmericanGut_ERP012803,1.37086955,0.026196879,52.329498,0.000000e+00,intercept
AmericanGut_ERP012803,0.17745217,0.010215155,17.371462,1.101611e-63,slope
Atlantic_ERP012887,1.31283050,0.018501000,70.959973,0.000000e+00,intercept
Atlantic_ERP012887,0.26358490,0.008496155,31.024021,8.138146e-179,slope
CanadaWater_ERP012927,1.64219066,0.017576526,93.430899,0.000000e+00,intercept
CanadaWater_ERP012927,0.24791751,0.008172616,30.335149,1.147952e-182,slope
Donna_ADSludge,1.38976953,0.059584235,23.324450,3.976050e-67,intercept
Donna_ADSludge,0.29108999,0.026836430,10.846822,4.698122e-23,slope
DuckWater_ERP012631,1.04481367,0.019282244,54.185274,0.000000e+00,intercept


In [86]:
fits_df_dgcat

study,samp_name,intercept_Estimate,intercept_Std. Error,intercept_t value,intercept_Pr(>|t|),slope_Estimate,slope_Std. Error,slope_t value,slope_Pr(>|t|)
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AmericanGut_ERP012803,ERR1090777,0.9423341,2.019023,0.46672787,0.6724991,0.02067529,0.3553608,0.05818113,0.9572629
AmericanGut_ERP012803,ERR1090778,-0.1456611,1.819209,-0.08006840,0.9412250,0.21850411,0.3201923,0.68241530,0.5439459
AmericanGut_ERP012803,ERR1090779,0.6975431,3.046331,0.22897814,0.8401695,0.13741896,0.5034295,0.27296563,0.8104821
AmericanGut_ERP012803,ERR1090780,0.1423014,2.130202,0.06680182,0.9509423,0.21507443,0.3749291,0.57364031,0.6063932
AmericanGut_ERP012803,ERR1090781,-0.3012734,2.368430,-0.12720380,0.9068264,0.27597225,0.4168588,0.66202813,0.5552656
AmericanGut_ERP012803,ERR1090782,0.2190173,1.731413,0.12649633,0.9073410,0.09398756,0.3047396,0.30841927,0.7779391
AmericanGut_ERP012803,ERR1090783,0.8171809,2.705659,0.30202653,0.7911448,0.09832956,0.4471310,0.21991220,0.8463452
AmericanGut_ERP012803,ERR1090784,0.3719168,2.327639,0.15978290,0.8832041,0.11315647,0.4096792,0.27620753,0.8003237
AmericanGut_ERP012803,ERR1090785,0.3775121,2.347939,0.16078443,0.8824803,0.11246579,0.4132522,0.27214806,0.8031638


# Metacommunity

In [56]:
#Read in the results from the NMGS runs 
metadf = read.table('./Results//metacommunity_diversity_20200601.txt', sep="\t", stringsAsFactors = F)

metadf <- metadf[,c("dirname", as.character(metabolic_energies$Metabolism))]

md <- melt(metadf, value.name = "theta", variable.name = "metabolism", measure.vars = metabolic_energies$Metabolism, )

theta_df <- merge(md, metabolic_energies, by.x = "metabolism", by.y="Metabolism")
#Drop missing values
theta_df <- theta_df[(!is.na(theta_df$theta)) & (theta_df$theta > 0),]

theta_df

,metabolism,dirname,theta,yield,dgcat,Name
,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<fct>
2,Acet_Methanogen,Atlantic_ERP012887,2.450817,0.067100,-31.0000,Acetoclastic Methanogen
3,Acet_Methanogen,CanadaWater_ERP012927,3.656222,0.067100,-31.0000,Acetoclastic Methanogen
4,Acet_Methanogen,Donna_ADSludge,7.817335,0.067100,-31.0000,Acetoclastic Methanogen
5,Acet_Methanogen,DuckWater_ERP012631,0.296142,0.067100,-31.0000,Acetoclastic Methanogen
7,Acet_Methanogen,Lise_Soils,1.882969,0.067100,-31.0000,Acetoclastic Methanogen
12,Acet_Methanogen,SevernTrent,2.949467,0.067100,-31.0000,Acetoclastic Methanogen
15,AerobicResp,AmericanGut_ERP012803,491.507611,13.421700,-2841.1997,Aerobic Heterotroph
16,AerobicResp,Atlantic_ERP012887,864.187471,13.421700,-2841.1997,Aerobic Heterotroph
17,AerobicResp,CanadaWater_ERP012927,805.506736,13.421700,-2841.1997,Aerobic Heterotroph


In [57]:
theta_fits <- theta_df %>% group_by(dirname) %>% do(data.frame(
    yield=summary(lm(log(theta) ~ log(yield), data=.))$coefficients,
    dgcat=summary(lm(log(theta) ~ log(-dgcat), data=.))$coefficients,stringsAsFactors = F))

theta_fits$param = c("intercept","slope")

In [62]:
theta_fitsdf<- melt(theta_fits, id.vars = c("dirname","param")) %>% dcast(dirname ~ param +variable)

In [63]:
theta_plot_yield <- ggplot(theta_df, aes(x=yield,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf, aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate), alpha=0.5, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies") 

theta_plot_dgcat <- ggplot(theta_df, aes(x=-dgcat,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf, aes(slope=slope_dgcat.Estimate, intercept=intercept_dgcat.Estimate), alpha=0.5, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression(-Delta*G[cat](kJmol^-1))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies") 


In [66]:
#Just oceans
theta_plot_yield <- ggplot(theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf[theta_fitsdf$dirname=="TARA_ERP001736",], aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate), alpha=1.0, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies") 

theta_plot_dgcat <- ggplot(theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=-dgcat,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf[theta_fitsdf$dirname=="TARA_ERP001736",], aes(slope=slope_dgcat.Estimate, intercept=intercept_dgcat.Estimate), alpha=1.0, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression(-Delta*G[cat](kJmol^-1))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies") 

ggsave('output_images/theta/TARA-yield.png',theta_plot_yield,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 
ggsave('output_images/theta/TARA-dgcat.png',theta_plot_dgcat,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 



In [67]:
theta_plot_yield <- ggplot(theta_df, aes(x=yield,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf[theta_fits$dirname=="TARA_ERP001736",], aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate), alpha=1.0, color="black") +
      
    
      geom_point(size=5, aes(color=metabolism), alpha=0.1) +
      #geom_point(data=theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta), color="black", size=5) +
    
      geom_point(size=2, color="LightBlue", alpha=0.1) + 

      #geom_point(data=theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta), color="black", size=5) +
      geom_point(data=theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta, color=metabolism), size=5) +
      geom_point(data=theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta), color="white", size=3) +
      geom_point(data=theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta, color=metabolism), size=1) +
      
      #geom_point(data=theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=yield,y=theta), color="red", size=2) +

      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies") 

theta_plot_dgcat <- ggplot(theta_df[theta_df$dirname == "TARA_ERP001736",], aes(x=-dgcat,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf[theta_fits$dirname=="TARA_ERP001736",], aes(slope=slope_dgcat.Estimate, intercept=intercept_dgcat.Estimate), alpha=1.0, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression(-Delta*G[cat](kJmol^-1))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies") 

ggsave('output_images/theta/TARA-full_yield_highlighted.png',theta_plot_yield,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 
ggsave('output_images/theta/TARA-full_dgcat.png',theta_plot_dgcat,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 



Warning message:
“Removed 2 rows containing missing values (geom_abline).”
Warning message:
“Removed 2 rows containing missing values (geom_abline).”


# Simple TARA

In [71]:
#Set up the plot axes - for log scales
xbreaks = c(seq(0.001,0.01,0.001),seq(0.01,0.1,0.01), seq(0.1,1,0.1), seq(1,10,1), seq(10,100,10), seq(100,1000,100),seq(1000,10000,1000))
xlabs <- c("0.001",rep("",9),"0.01",rep("",9),"0.1",rep("",9),"1",rep("",9), "10", rep("",9), "100", "","","","500","","","","", "1000", "", rep("",9))

ybreaks = c(seq(0.1,1,0.1),seq(1,10,1), seq(10,100,10), seq(100,1000,100), seq(1000,10000,1000))
ylabs <- c("0.1",rep("",9),"1",rep("",9),"10","20","30","40","50","60","70","80","90","100","",rep("",9),"1000",rep("",9))

In [70]:
theta_fits <- theta_df %>% group_by(dirname) %>% filter(C.Source.type=="S") %>% do(data.frame(
    yield=summary(lm(log(theta) ~ log(yield), data=.))$coefficients,
    dgcat=summary(lm(log(theta) ~ log(-dgcat), data=.))$coefficients,stringsAsFactors = F))

theta_fits$param = c("intercept","slope")
rownames(theta_fitsdf_yield) <- theta_fitsdf_yield$dirname




theta_fitsdf<- melt(theta_fits, id.vars = c("dirname","param")) %>% dcast(dirname ~ param +variable)

theta_plot_yield <- ggplot(theta_df[(theta_df$dirname=="TARA_ERP001736") & (theta_df$C.Source.type=="S"),], aes(x=yield,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf[theta_fitsdf$dirname=="TARA_ERP001736",], aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate), alpha=1.0, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs, limits = c(5,30)) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nTARA dataset- simple carbon substrates") 

theta_plot_dgcat <- ggplot(theta_df[(theta_df$dirname=="TARA_ERP001736") & (theta_df$C.Source.type=="S"),], aes(x=-dgcat,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=theta_fitsdf[theta_fitsdf$dirname=="TARA_ERP001736",], aes(slope=slope_dgcat.Estimate, intercept=intercept_dgcat.Estimate), alpha=1.0, color="black") +
      
      geom_point(size=5, aes(color=metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs, limits = c(5,30)) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +

      xlab(expression(-Delta*G[cat](kJmol^-1))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nTARA dataset - simple carbon substrates") 

ggsave('output_images/theta/simple_carbon_TARA-yield.png',theta_plot_yield,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 
ggsave('output_images/theta/simple_carbon_TARA-dgcat.png',theta_plot_dgcat,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 

theta_plot_dgcat

theta_plot_yield

ERROR: Error: object 'C.Source.type' not found


In [ ]:
biome_theta_df = merge(theta_df, study_info_df, by.x="dirname", by.y="study")

biome_theta_fits <- biome_theta_df %>% group_by(biome) %>% do(data.frame(
    yield=summary(lm(log(theta) ~ log(yield), data=.))$coefficients,
    dgcat=summary(lm(log(theta) ~ log(-dgcat), data=.))$coefficients,stringsAsFactors = F))

biome_theta_fits$param = c("intercept","slope")
#rownames(biome_theta_fits) <- biome_theta_fits$biome

biome_theta_fitsdf<- melt(biome_theta_fits, id.vars = c("biome","param")) %>% dcast(biome ~ param +variable)

In [ ]:
biome_theta_fitsdf[c("biome","slope_yield.Estimate","slope_yield.Std..Error", "slope_dgcat.Pr...t..")]

In [ ]:
mdf_casted <- dcast(biome_theta_df[,c("metabolism", "theta", "biome")], metabolism ~ biome, value.var = "theta", fun.aggregate = mean)
mdf_casted2 <-dcast(biome_theta_df[,c("metabolism", "theta", "dirname")] , metabolism ~ dirname, value.var = "theta")


In [ ]:
library(Skillings.Mack)


In [ ]:
t(as.matrix(mdf_casted2[,-1]))

In [ ]:
sk_res <- Ski.Mack(as.matrix(1/mdf_casted[,-1]),simulate.p.value = T, B=100 )

In [ ]:
sk_res

In [ ]:
biome_theta_fitsdf

In [ ]:
theta_plot_yield <- ggplot(biome_theta_df, aes(x=yield,y=theta)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=biome_theta_fitsdf, aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate, color=biome),
                  alpha=1.0, size=2) +
      
      #geom_point(size=5, aes(color=biome), alpha=1.0) +
      geom_point(size=2, color="black", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = biome_color_list, name="Source biome" ) +

      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab(expression(theta ~ "(metacommunity diversity)"))+
      
      
      ggtitle("Metacommunity diversity:\nall studies by biome") 
theta_plot_yield

In [ ]:
anova(lm(log(theta) ~ log(yield)*biome, biome_theta_df))


In [ ]:
biome_theta_fitsdf[,c(1,2,5,10,13)][c(1,4,5,2,3)]

In [ ]:
summary(lm(log(theta) ~ log(yield):biome + biome + yield , biome_theta_df))

# Phylogenetic diversity

In [37]:
#Load the tree
gg_tree = read_tree_greengenes('Inputs/gg_13_5_otus_99_annotated.tree')
knowntips = gg_tree$tip.label

#load the database entries
gg_dat = read.table('Inputs/gg_13_5_taxonomy.txt', sep="\t", header=F, stringsAsFactors = F)
colnames(gg_dat) <- c("taxid","taxstr")
tree_dat = gg_dat[gg_dat$taxid %in% knowntips,]

In [38]:
m = match(gg_tree$tip.label, rownames(tree_dat))
prunetips = gg_tree$tip.label[is.na(head(m))]
length(prunetips)

[1] 169543

In [39]:
taxdf = as.data.frame(t(as.data.frame(strsplit(gsub('.__', '', as.character(tree_dat$taxstr)), ';'), stringsAsFactors=FALSE)), stringsAsFactors=FALSE)
colnames(taxdf) <- c("Kingdom","Phylum","Class","Order","Family","Genus","Species")
taxdf$taxid <- tree_dat$taxid
rownames(taxdf) <- taxdf$taxid

In [40]:
cleantaxfs = sapply(taxdf,function(x) gsub(' ','',x))

In [41]:
cleandf = data.frame(cleantaxfs, stringsAsFactors = F)
rownames(cleandf) = rownames(taxdf)

In [42]:
#Get lists of tip labels we need to search for
tax_groups = lapply(Taxa_Search_Expressions, function(x) rownames(subset(cleandf, eval(x))))

In [43]:
sapply(tax_groups, length)
sum(sapply(tax_groups, length))

AerobicResp Ammonia_Oxidizer    FeIII_reducer             Ferm 
           44417              615              643            64945 
     Fe_oxidizer    H2_Methanogen      H2_Oxidizer            Mixed 
              66              261              210             2567 
     CO_Oxidizer     Methanotroph  Acet_Methanogen   CH3_Methanogen 
               4              844              218               38 
        N2_Fixer              NOB              NRB              SRB 
               6              990               12             4313 
    Ferm_oxalate     Ferm_peptone            Photo           SRB_S0 
              18               24             9220                3 
             SOB           SRB_H2           ANAMOX 
             475               20               77

[1] 129986

In [44]:
ul = unique(unlist(tax_groups))
m = match(gg_tree$tip.label, ul, )
prunelist = gg_tree$tip.label[is.na(m)]

In [45]:
nvec=names(tax_groups)
names(nvec)=nvec
all_prune_taxa = lapply(nvec, function(x) unique(unlist(tax_groups[-which(nvec == x)])))

In [46]:
working_tree = drop.tip(gg_tree, prunelist)

In [47]:
all_trees = lapply(all_prune_taxa, function(x) drop.tip(working_tree,x))

Warning message in drop.tip(working_tree, x):
“drop all tips of the tree: returning NULL”
Warning message in drop.tip(working_tree, x):
“drop all tips of the tree: returning NULL”


In [48]:
pd_list = lapply(all_trees, function(x) sum(x$edge.length))
pd_df = as.data.frame(t(data.frame(pd_list)))
pd_df[,2] = rownames(pd_df)
colnames(pd_df) <- c("pd","metab")

In [49]:
fulldf = merge(metabolic_energies, pd_df, by.x="Metabolism", by.y="metab")
pd_fulldf <- fulldf[order(fulldf$yield, decreasing = T),]

In [50]:
pd_fulldf 

,Metabolism,yield,dgcat,Name,pd
,<fct>,<dbl>,<dbl>,<fct>,<dbl>
2,AerobicResp,13.421700,-2841.19969,Aerobic Heterotroph,427.68770
10,Methanotroph,1.259100,-813.33328,Methanotroph,11.60265
7,Ferm,0.986400,-823.83328,Fermenting Heterotroph,1010.52461
6,FeIII_reducer,0.724200,-809.39953,Iron III reducer,8.65103
12,SOB,0.697100,-796.63328,Sulphur oxidizer,5.57092
9,H2_Oxidizer,0.223200,-237.20000,Hydrogen oxidizer,0.11594
5,CH3_Methanogen,0.127200,-78.63332,Methylotrophic Methanogen,0.88821
4,ANAMOX,0.104934,-362.80000,ANAMOX,1.04300
13,SRB,0.103300,-47.70000,Sulphate reducer,59.71780


In [51]:
pd_fits <- pd_fulldf %>% filter(pd > 0.0) %>% do(data.frame(
        yield=summary(lm(log(pd) ~ log(yield), data=.))$coefficients,
        dgcat=summary(lm(log(pd) ~ log(-dgcat), data=.))$coefficients,stringsAsFactors = F))

pd_fits$param = c("intercept","slope")

pd_fitsdf<- melt(pd_fits, id.vars = c("param")) %>% dcast(. ~ param +variable)



pd_plot_yield <- ggplot(pd_fulldf, aes(x=yield,y=pd)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=pd_fitsdf, aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate), 
                  alpha=1.0, color="black",size=2) +
      
      geom_point(size=5, aes(color=Metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      #scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +
      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +
      #scale_color_manual(values = c_src_colours, labels = c_src_labels, name="Anabolic C source" ) +
      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab("Phylogenetic diversity (Faith's PD)")+
      
      
      ggtitle("Phylogenetic diversity:\nMetabolism")


pd_plot_dgcat <- ggplot(pd_fulldf, aes(x=-dgcat,y=pd)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=pd_fitsdf, aes(slope=slope_dgcat.Estimate, intercept=intercept_dgcat.Estimate),
                  alpha=1.0, color="black", size=2) +
      
      geom_point(size=5, aes(color=Metabolism), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +
      #scale_color_manual(values = c_src_colours, labels = c_src_labels, name="Anabolic C source" ) +
      #scale_color_manual(labels = c(C="Complex", S = "Simple"), name="Anabolic C source" ) +
      #xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      xlab(expression(-Delta*G[cat] (kJmol^{-1}))) +
      ylab("Phylogenetic diversity (Faith's PD)")+
      
      ggtitle("Phylogenetic diversity:\nMetabolism") 

ggsave(paste('output_images/phylogeny/PD-dgcat-metabolisms-.png', sep=""),pd_plot_dgcat,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 
ggsave(paste('output_images/phylogeny/PD-yield-metabolisms-.png', sep=""),pd_plot_yield,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 


In [52]:
pd_fits <- pd_fulldf %>% filter(pd > 0.0) %>% group_by(C.Source.type) %>% do(data.frame(
        yield=summary(lm(log(pd) ~ log(yield), data=.))$coefficients,
        dgcat=summary(lm(log(pd) ~ log(-dgcat), data=.))$coefficients,stringsAsFactors = F))

pd_fits$param = c("intercept","slope")

pd_fitsdf<- melt(pd_fits, id.vars = c("C.Source.type","param")) %>% dcast(C.Source.type ~ param +variable)



pd_plot_yield <- ggplot(pd_fulldf, aes(x=yield,y=pd)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=pd_fitsdf, aes(slope=slope_yield.Estimate, intercept=intercept_yield.Estimate), alpha=0.5, color="black") +
      
      geom_point(size=5, aes(color=C.Source.type), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      #scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +
      #scale_color_manual(values = metab_color_list, labels = metab_label_list, name="Metabolic classification" ) +
      scale_color_manual(values = c_src_colours, labels = c_src_labels, name="Anabolic C source" ) +
      xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      ylab("Phylogenetic diversity (Faith's PD)")+
      
      
      ggtitle("Phylogenetic diversity:\nCarbon source") 


pd_plot_dgcat <- ggplot(pd_fulldf, aes(x=-dgcat,y=pd)) + theme_bw(base_size = 18, base_family = "arial") +
 
      geom_abline(data=pd_fitsdf, aes(slope=slope_dgcat.Estimate, intercept=intercept_dgcat.Estimate), alpha=0.5, color="black") +
      
      geom_point(size=5, aes(color=C.Source.type), alpha=1.0) +
      geom_point(size=2, color="LightBlue", alpha=0.8) + 
      scale_x_continuous(trans = "log", breaks=xbreaks, labels=xlabs) + 
      scale_y_continuous(trans = "log", breaks=ybreaks, labels=ylabs) +  

      scale_color_manual(values = c_src_colours, labels = c_src_labels, name="Anabolic C source" ) +
      #scale_color_manual(labels = c(C="Complex", S = "Simple"), name="Anabolic C source" ) +
      #xlab(expression("Growth yield : " ~ bgroup("(", over(-Delta*G[cat],Delta*G[ana]+Delta*G[dis]),")"))) +
      xlab(expression(-Delta*G[cat] (kJmol^{-1}))) +
      ylab("Phylogenetic diversity (Faith's PD)")+
      
      
      ggtitle("Phylogenetic diversity:\nCarbon source") 


ggsave(paste('output_images/phylogeny/PD-dgcat-c-src-.png', sep=""),pd_plot_dgcat,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 
ggsave(paste('output_images/phylogeny/PD-yield-c-src-.png', sep=""),pd_plot_yield,dpi = 600,units="mm", width = 150*2, height = 100*2, limitsize = FALSE) 




ERROR: Error: Column `C.Source.type` is unknown
